# L5: Parameter-efficient fine-tuning

Fine-tuning all parameters of pre-trained language models can be resource-intensive. Because of this, current research in natural language processing is looking into developing methods for adapting models to downstream tasks without full fine-tuning. These methods only tune a small number of model parameters while yielding performance comparable to that of a fully fine-tuned model.

In this lab, you will implement LoRA, one of the most well-known methods for parameter-efficient fine-tuning. LoRA stands for “Low-Rank Adaptation of Large Language Models” and was originally described in a research article by [Hu et al. (2021)](https://arxiv.org/abs/2106.09685).

Along the way, you will earn experience with [Hugging Face Transformers](https://huggingface.co/docs/transformers/en/index), a state-of-the-art library for training and deploying language models, as well as with several related libraries. In particular, you will learn a best-practice workflow for downloading a Transformer model and fine-tuning it on the downstream task of binary sentiment classification.

## Dataset

The data for this lab comes from the [Large Movie Review Dataset](https://ai.stanford.edu/~amaas/data/sentiment/). The full dataset consists of 50,000 highly polar movie reviews collected from the Internet Movie Database (IMDB). Here, we use a random sample consisting of 2,000 reviews for training and 500 reviews for evaluation.

To load the dataset, we use the [Hugging Face Datasets](https://huggingface.co/docs/datasets/en/index) library.

In [65]:
from torch import nn

In [66]:
from datasets import load_dataset

imdb_dataset = load_dataset('csv', data_files = {'train': 'train.csv', 'eval': 'eval.csv'})

imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'review', 'label'],
        num_rows: 2000
    })
    eval: Dataset({
        features: ['index', 'review', 'label'],
        num_rows: 500
    })
})

As we can see, each sample in the dataset is a record with three fields: an internal index (`index`, an integer), the text of the review (`review`, a string), and the sentiment label (`label`, an integer – 1&nbsp;for “positive” and 0&nbsp;for “negative” sentiment).

Here is an example record:

In [ ]:
imdb_dataset['train'][645]

{'index': 2981,
 'review': 'Brilliant execution in displaying once and for all, this time in the venue of politics, of how "good intentions do actually pave the road to hell". Excellent!',
 'label': 1}

## Tokeniser

As our pre-trained language model, we will use [DistilBERT](https://huggingface.co/docs/transformers/en/model_doc/distilbert), a compact encoder model with 40% less parameters than BERT base. DistilBERT is not actually a *large* language model by modern standards and thus does not benefit as much from parameter-efficient fine-tuning as other models. However, it has the benefit of being light and fast, and can be run even on consumer hardware.

To feed the movie reviews to DistilBERT, we need to tokenise them and encode the resulting tokens as integers in the model vocabulary. We start by loading the DistilBERT tokeniser using the [Auto classes](https://huggingface.co/docs/transformers/en/model_doc/auto):

In [68]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

We then create a tokenised version of the dataset:

In [69]:
def tokenize_function(batch):
    return tokenizer(batch['review'], padding=True, truncation=True)

tokenized_imdb_dataset = imdb_dataset.map(tokenize_function, batched=True)

tokenized_imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    eval: Dataset({
        features: ['index', 'review', 'label', 'input_ids', 'attention_mask'],
        num_rows: 500
    })
})

As we can see, tokenising adds two additional fields to each review: `input_ids` is the list of token ids corresponding to the review, and `attention_mask` is the list of indices specifying which tokens the encoder should attend to.

To avoid trouble when fine-tuning the model later, the next cell disables tokeniser parallelism.

In [70]:
import os

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

## Trainer

In this section, we will set up our workflow for training and evaluating DistilBERT models. The central component in this workflow is the [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer), which provides extensive configuration options. Here, we leave most of these options at their default value. Two changes we *do* make are to enable evaluation of the trained model after each epoch, and to log the training and evaluation loss after every 5&nbsp;training steps (the default is 500).

In [71]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='tmp_trainer',
    evaluation_strategy='epoch',
    logging_steps=5,
)

In addition to the loss, we also track classification accuracy. For this we import the [Hugging Face Evaluate](https://huggingface.co/docs/evaluate/en/index) library and define a small helper function `compute_metrics()` that the trainer will call after each epoch.

In [72]:
import evaluate

accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In the next cell we define a convenience function `make_trainer()` that creates a readily-configured trainer for a specified model (*model*). We will use this trainer both to train the model on the training section of the tokenised review dataset, and to evaluate it on the evaluation section.

In [73]:
from transformers import Trainer

def make_trainer(model):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_imdb_dataset['train'],
        eval_dataset=tokenized_imdb_dataset['eval'],
        compute_metrics=compute_metrics,
    )
    return trainer

## Problem 1: Full fine-tuning (1 point)

In the rest of this notebook, we will work our way to the implementation of LoRA, and compare LoRA to traditional fine-tuning methods. Our first point of reference is a fully fine-tuned DistilBERT model.

We start by loading the pre-trained model:

In [74]:
from transformers import AutoModelForSequenceClassification

pretrained_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

pretrained_model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

The architecture of DistilBERT is that of a standard Transformer encoder with an embedding layer (`embeddings`) followed by a stack of six Transformer blocks (`transformer`) and a feedforward network with two linear layers (`pre_classifier` and `classifier`) and a final dropout layer (`dropout`).

### Count the number of trainable parameters

One relevant measure in the context of parameter-efficient fine-tuning is the number of parameters that need to be changed when training a model. Your first task in this lab is to write a function `num_trainable_parameters()` that calculates this number for a given model.

In [80]:
def num_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

The function should implement the following specification:

> **num_trainable_parameters** (*model*)
>
> Returns the number of float-valued trainable parameters in the specified *model* as an integer.

#### 👍 Hint

The term *parameter* can refer to either complete tensors or the individual elements of these tensors. For example, a linear layer created by `nn.Linear(3, 5)` has 2&nbsp;tensor-valued parameters (a weight matrix and a bias vector) and 20&nbsp;float-valued parameters (the elements of these tensors). To get the tensor-valued parameters of a model, you can use the [`parameters()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters) method. A parameter is *trainable* if it requires gradient.

#### 🤞 Test your code

To test your code, apply your function to the pre-trained model. The correct number of float-valued trainable parameters for this model is 66,955,010.

In [76]:
num_trainable_parameters(pretrained_model)

66955010

### Fine-tuning

When we load the pre-trained model, the Hugging Face Transformers library warns us that the weights of the feedforward network have not yet been trained. To do so, in the next cell, we pass the pre-trained model to a trainer and initiate the fine-tuning process.

**⚠️ Please note that fine-tuning the model will take some time! ⚠️**

You can work on the other problems in this lab while you are waiting.

In [ ]:
finetuned_trainer = make_trainer(pretrained_model)

finetuned_trainer.train()

Because full fine-tuning is so resource-intensive, we save the fine-tuned model to disk:

In [ ]:
finetuned_trainer.save_model('finetuned')

Later in this notebook, whenever you need the fully fine-tuned version of the model, you can load it as follows:

In [149]:
finetuned_model = AutoModelForSequenceClassification.from_pretrained('finetuned')

### Convenience functions

Because we will repeat the steps we just took to fine-tune the pre-trained model several times in this notebook, we define two convenience functions:

In [22]:
def train(model):
    print('Number of trainable parameters:', num_trainable_parameters(model))
    trainer = make_trainer(model)
    trainer.train()
    return model

In [97]:
def evaluate(model):
    trainer = make_trainer(model)
    return trainer.evaluate()

## Problem 2: Tuning the final layers only (1 point)

If full fine-tuning marks one end of the complexity spectrum, the other end is marked by only tuning the final layers of the transformer – the *head* of the model. In the case of DistilBERT, the head consists of the `pre_classifier` and `classifier` layers.

Implement the head-tuning strategy by coding the following function:

In [96]:
def make_headtuned_model():
    headtuned_model: AutoModelForSequenceClassification = (
        AutoModelForSequenceClassification.from_pretrained(
            "distilbert-base-uncased", num_labels=2
        )
    )  # Load the pretrained model
    print(dir(headtuned_model))
    # Freeze all the parameters of the base model
    for parameter in headtuned_model.base_model.parameters():
        parameter.requires_grad = False

    # Unfreeze the parameters of the head
    for parameter in headtuned_model.classifier.parameters():
        parameter.requires_grad = True

    headtuned_model = train(headtuned_model)
    return headtuned_model

Here is the specification of this function:

> **make_headtuned_model** ()
>
> Returns a model that is identical to the pre-trained model, except that the head layers have been trained on the sentiment data. (The other parameters of the pre-trained model are left untouched.)

#### 👍 Hint

You freeze a parameter by setting its `requires_grad`-attribute to `False`.

Once you have an implementation of the head-tuning strategy, evaluate it on the evaluation data. How much accuracy do we lose when only training the final layers of the pre-trained model, compared to full fine-tuning?

In [ ]:
headtuned_model = make_headtuned_model()

#### 🤞 Test your code

If you configured your model correctly, `num_trainable_parameters()` should show 592,130 trainable parameters.

For future reference, we also save the head-tuned model:

In [94]:
make_trainer(headtuned_model).save_model('headtuned')

## Problem 3: Layer surgery (4 points)

LoRA works by “wrapping” frozen layers from the pre-trained Transformer model inside adapter modules. Conventionally, this wrapping is only applied to the linear layers that transform the queries and values in the self-attention mechanism. To implement the wrapping, we need functions to extract and replace layers in a model. Your task in this section is to code these functions.

### Extracting layers

Code a function that extracts the query and value linear layers from a DistilBERT model:

In [131]:
def extract(model) -> dict[str, nn.Linear]:
    """Extract the parameters of the model."""
    output: dict[str, nn.Linear] = {}
    for i in range(model.base_model.config.n_layers):
        key_name_q = f"base_model.transformer.layer.{i}.attention.q_lin"
        key_name_v = f"base_model.transformer.layer.{i}.attention.v_lin"
        output[key_name_q] = model.get_submodule(key_name_q)
        output[key_name_v] = model.get_submodule(key_name_v)
    return output

Implement this function to match the following specification:

> **extract** (*model*)
>
> Takes a DistilBERT model (*model*) and extracts the query and value linear layers from each block of the Transformer. Returns a dictionary mapping the DistilBERT module names of these layers to the layers themselves (instances of `nn.Linear`).

#### 👍 Hint

As we saw earlier, the DistilBERT model consists of a hierarchy of nested submodules. Each of these can be addressed by a fully-qualified string name. Use [`get_submodule()`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.get_submodule) to retrieve a layer by name. You can hard-wire the names of the layers you want to extract.

#### 🤞 Test your code

To test your code, check the number of trainable float-valued parameters in the extracted layers. This number should be 7,087,104.

In [150]:
headtuned_model = AutoModelForSequenceClassification.from_pretrained('headtuned')

In [127]:
extracted = extract(headtuned_model)
""" print(extracted)
values = list(extracted.values())
print(len(values))
print(values)
for layer in values:
  print(num_trainable_parameters(layer))
  for parameter in layer.parameters():
    print(parameter.requires_grad) """


print(sum([num_trainable_parameters(layer) for layer in extract(headtuned_model).values()]))

7087104


### Replacing layers

Next, code the inverse of the `extract()` function to replace selected layers of a module using a dictionary of named layers.

In [157]:
def replace(model: nn.Module, named_layers: dict):
    """
    Replace a submodule within a model at a specified path with a new module.

    :param model: The model containing the submodule to replace.
    :param named_layers: A dictionary containing the new modules to replace the old ones
    """

    for key, value in named_layers.items():


        # Split the path and separate the target layer from its parent path
        path_parts = key.split(".")
        submodule_path, target_layer_name = ".".join(path_parts[:-1]), path_parts[-1]

        # Traverse to the parent module of the target layer
        parent_module = model
        for part in submodule_path.split("."):
                parent_module = parent_module.get_submodule(part)

        # Replace the target layer within its parent module
        setattr(parent_module, target_layer_name, value)
    return model

Implement this function to match the following specification:

> **replace** (*model*, *named_layers*)
>
> Takes a DistilBERT model (*model*) and a dictionary in the format returned by `extract()` (*named_layers*) and injects the extracted layers into the model. More specifically, suppose that *named_layers* contains a key–value pair `(name, layer)`. Then the function replaces the submodule of *model* addressed by the fully-qualified string name `name` by the layer `layer`. Returns the modified model.

#### 👍 Hint

Use [`getattr()`](https://docs.python.org/3/library/functions.html#getattr) and [`setattr()`](https://docs.python.org/3/library/functions.html#setattr) to return or set the value of a named submodule.

#### 🤞 Test your code

To test your implementation, write code that (1)&nbsp;extracts the query and value linear layers from the fine-tuned model; (2)&nbsp;replaces these layers with clones with random weights; and (3)&nbsp;replaces these layers again with the original versions. Evaluating the modified model after step&nbsp;(2) should yield a near-random accuracy. Evaluating it again after step&nbsp;(3) should yield the original accuracy.

The following function should be helpful. It clones a linear layer, copying the weights and the bias from the original.

In [141]:
import torch.nn as nn

def clone_linear(original):
    out_features, in_features = original.weight.shape
    copy = nn.Linear(in_features, out_features)
    copy.load_state_dict(original.state_dict())
    return copy

In [158]:
finetuned_model = AutoModelForSequenceClassification.from_pretrained('finetuned')

# 1. extract the query and value linear layers from the fine-tuned model
extracted = extract(finetuned_model)

# 2. clone the query and value linear layers
cloned = {key: clone_linear(value) for key, value in extracted.items()}

# 3. replace the weights of the cloned layers with random weights
for key, value in cloned.items():
    nn.init.normal_(value.weight)
    nn.init.zeros_(value.bias)

print("model before replacing")
print(evaluate(finetuned_model))
# print(headtuned_model)
# 4. replace the model layers with the cloned layers
finetuned_model = replace(finetuned_model, cloned)
print("model after replacing")
print(evaluate(finetuned_model))
# print(headtuned_model)

# 5. replace the cloned layers with the original layers
finetuned_model = replace(finetuned_model, extracted)
# print(headtuned_model)
print("model after replacing the cloned layers with the original layers")
print(evaluate(finetuned_model))

model before replacing
































































































































100%|██████████| 63/63 [00:27<00:00,  2.33it/s]


{'eval_loss': 0.47883087396621704, 'eval_accuracy': 0.894, 'eval_runtime': 27.5476, 'eval_samples_per_second': 18.15, 'eval_steps_per_second': 2.287}
model after replacing
































































































































100%|██████████| 63/63 [00:27<00:00,  2.31it/s]


{'eval_loss': 0.7002705931663513, 'eval_accuracy': 0.504, 'eval_runtime': 27.6884, 'eval_samples_per_second': 18.058, 'eval_steps_per_second': 2.275}
model after replacing the cloned layers with the original layers
































































































































100%|██████████| 63/63 [00:27<00:00,  2.32it/s]

{'eval_loss': 0.47883087396621704, 'eval_accuracy': 0.894, 'eval_runtime': 27.5944, 'eval_samples_per_second': 18.12, 'eval_steps_per_second': 2.283}


## Problem 4: Low-rank approximation (6 points)

The basic idea behind LoRA is to conceptualise fine-tuned weights as a sum $W_0 + \Delta W$ of the weights from the pre-trained model, $W_0$, and a low-rank update matrix $\Delta W$. The goal of fine-tuning, then, is to learn the update matrix; this happens in the adapter layers.

Before we get to the implementation of the LoRA adapter layers in Problem&nbsp;4, in this section, we first check to what extent the assumption that fine-tuning can be described by low-rank matrices holds true for DistilBERT. To do so, we will “cheat” and replace the query and value linear layers of the head-tuned model with low-rank approximations. The technical key to this is the truncated singular value decomposition (SVD).

### Low-rank matrix approximation

Your first task in this section is to implement the low-rank matrix approximation.

In [164]:
import torch
def approximate(matrix: torch.Tensor, rank: int) -> torch.Tensor:
    U,S,Vh = torch.linalg.svd(matrix)
    U = U[:, :rank]
    S = torch.diag(S[:rank])
    Vh = Vh[:rank, :]
    approx = U @ S @ Vh
    assert isinstance(approx, torch.Tensor)
    return approx

Implement this function to match the following specification:

> **approximate** (*matrix*, *rank*)
>
> Takes a 2D-tensor (*matrix*) and an integer rank $r$ (*rank*), computes the truncated SVD with rank $r$ on the tensor, and returns the corresponding low-rank approximation matrix.

#### 👍 Hint

If you need a refresher on the low-rank matrix approximation, read the corresponding section from the Wikipedia article on the [Singular value decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition#Low-rank_matrix_approximation). The truncated SVD is an extension of the full SVD; the latter can be computed using [`torch.linalg.svd()`](https://pytorch.org/docs/stable/generated/torch.linalg.svd.html).

#### 🤞 Test your code

To test your code, run the following cell. It creates a matrix `original` with rank $r \leq 8$ and after that the rank-$8$ approximation matrix `approximation`. You should find that the distance between the two matrices is very low.

In [165]:
original = torch.rand(768, 8) @ torch.rand(8, 384)
approximation = approximate(original, 8)
torch.dist(original, approximation)

tensor(0.0006)

### Approximated fine-tuned model (version 1)

In the next step, your task is to construct a version of the head-tuned model in which every query and value linear layer is replaced by a low-rank approximation of the corresponding layer from the fully fine-tuned model.

In [175]:
def make_approximated_model_1(rank: int) -> nn.Module:
    """ Create a new head-tuned model with the query and value linear layers approximated to a given rank. """    
    finetuned_model = AutoModelForSequenceClassification.from_pretrained('finetuned')
    extracted = extract(finetuned_model)
    for key, value in extracted.items():
        value.weight = nn.Parameter(approximate(value.weight, rank))
        extracted[key] = value
    finetuned_model = replace(finetuned_model, extracted)
    return finetuned_model

Here is the specification of this function:

> **make_approximated_model_1** (*rank*)
>
> Takes an integer rank $r$ (*rank*) and returns a version of the head-tuned model in which every query and value linear layer is replaced by its $r$-approximated corresponding layer from the fully fine-tuned model.

Run the next cell to evaluate your model for different rank values. Start with the full rank and then halve the rank in each step. What is the lowest rank that still gives you a higher accuracy than the head-tuned model?

In [178]:
headtuned_model = AutoModelForSequenceClassification.from_pretrained('headtuned')
evaluate(headtuned_model)






























































































































































































100%|██████████| 63/63 [00:27<00:00,  2.29it/s]


{'eval_loss': 0.5080119967460632,
 'eval_accuracy': 0.818,
 'eval_runtime': 28.3503,
 'eval_samples_per_second': 17.637,
 'eval_steps_per_second': 2.222}

In [179]:
rank = 768
while rank > 0:
    approximated_model = make_approximated_model_1(rank)
    print(rank, evaluate(approximated_model))
    rank //= 2






























































































































































































100%|██████████| 63/63 [00:27<00:00,  2.27it/s]


768 {'eval_loss': 0.4788309633731842, 'eval_accuracy': 0.894, 'eval_runtime': 28.2707, 'eval_samples_per_second': 17.686, 'eval_steps_per_second': 2.228}































































































































































































100%|██████████| 63/63 [00:27<00:00,  2.27it/s]


384 {'eval_loss': 0.4604506492614746, 'eval_accuracy': 0.898, 'eval_runtime': 28.3093, 'eval_samples_per_second': 17.662, 'eval_steps_per_second': 2.225}































































































































































































100%|██████████| 63/63 [00:27<00:00,  2.28it/s]


...

6 {'eval_loss': 0.682673454284668, 'eval_accuracy': 0.604, 'eval_runtime': 27.5076, 'eval_samples_per_second': 18.177, 'eval_steps_per_second': 2.29}































































































































































































100%|██████████| 63/63 [00:26<00:00,  2.35it/s]


3 {'eval_loss': 0.6882462501525879, 'eval_accuracy': 0.582, 'eval_runtime': 27.2919, 'eval_samples_per_second': 18.32, 'eval_steps_per_second': 2.308}































































































































































































100%|██████████| 63/63 [00:27<00:00,  2.30it/s]

1 {'eval_loss': 0.6979635953903198, 'eval_accuracy': 0.474, 'eval_runtime': 27.8859, 'eval_samples_per_second': 17.93, 'eval_steps_per_second': 2.259}


### Approximated fine-tuned model (version 2)

In the approximated model from the previous section, the truncated SVD is applied to the full weight matrix of the fine-tuned model: $W_0 + \Delta W$. In LoRA, the low-rank approximation only applies to the *update matrix* $\Delta W$, i.e., the difference between the fully fine-tuned weights and the pre-trained weights.

In [180]:
def make_approximated_model_2(rank):
    # Extract the layers from the head-tuned model
    headtuned_model = AutoModelForSequenceClassification.from_pretrained("headtuned")
    headtuned_layers = extract(headtuned_model)

    # Extract the layers from the fine-tuned model
    finetuned_model = AutoModelForSequenceClassification.from_pretrained("finetuned")
    finetuned_layers = extract(finetuned_model)

    # Compute the new layers
    tmp = {}
    for name, layer in headtuned_layers.items():
        clone = clone_linear(layer)

        with torch.no_grad():
            a = headtuned_layers[name].weight.data  # original pretrained weights
            b = finetuned_layers[name].weight.data  # fine-tuned weights

            delta = (
                b - a
            )  # difference between the fine-tuned and original weights delta W

            clone.weight.copy_(a + approximate(delta, rank))

        tmp[name] = clone

    return replace(headtuned_model, tmp)

Implement the function to match the following specification:

> **make_approximated_model_2** (*rank*)
>
> Takes an integer rank $r$ (*rank*) and returns a version of the head-tuned model in which the weight matrix of every query and value linear layer is replaced by the sum $W_0 + \Delta W$, where $W_0$ is the weight matrix of the pre-trained model and $\Delta W$ is the rank-$r$ approximation of the update matrix, i.e., the difference between the fully fine-tuned weights and the pre-trained weights.

Run the next cell to evaluate your model for different rank values. Start with the rank from the approximated model from the previous section and then halve the rank in each step. What is the lowest rank that still gives you a higher accuracy than the head-tuned model?

In [182]:
rank = 768
while rank > 0:
    approximated_model_2 = make_approximated_model_2(rank)
    print(rank, evaluate(approximated_model_2))
    rank //= 2






























































































































































































100%|██████████| 63/63 [00:27<00:00,  2.28it/s]


768 {'eval_loss': 0.3571978807449341, 'eval_accuracy': 0.866, 'eval_runtime': 28.683, 'eval_samples_per_second': 17.432, 'eval_steps_per_second': 2.196}































































































































































































100%|██████████| 63/63 [00:27<00:00,  2.26it/s]


384 {'eval_loss': 0.3572976589202881, 'eval_accuracy': 0.866, 'eval_runtime': 28.4919, 'eval_samples_per_second': 17.549, 'eval_steps_per_second': 2.211}































































































































































































100%|██████████| 63/63 [00:27<00:00,  2.27it/s]


...

6 {'eval_loss': 0.3735736310482025, 'eval_accuracy': 0.864, 'eval_runtime': 27.7094, 'eval_samples_per_second': 18.044, 'eval_steps_per_second': 2.274}































































































































































































100%|██████████| 63/63 [00:27<00:00,  2.27it/s]


3 {'eval_loss': 0.38590219616889954, 'eval_accuracy': 0.85, 'eval_runtime': 28.47, 'eval_samples_per_second': 17.562, 'eval_steps_per_second': 2.213}































































































































































































100%|██████████| 63/63 [00:28<00:00,  2.25it/s]

1 {'eval_loss': 0.45019596815109253, 'eval_accuracy': 0.84, 'eval_runtime': 28.8404, 'eval_samples_per_second': 17.337, 'eval_steps_per_second': 2.184}


In [ ]:
approximated_model_2 = make_approximated_model_2(3)

evaluate(approximated_model_2)

## Problem 5: LoRA (4 points)

In this section, you will implement the LoRA adapters and fine-tune the adapted model.

### Implement the adapter

A LoRA adapter implements the forward function

$$
y = x W_0 + x \Delta W = x W_0 + x A B
$$

where $W_0$ is a linear transformation from the pre-trained model and $\Delta W$ is a learned update matrix, deconstructed into the product $AB$ of two rank-$r$ matrices $A$ and $B$. LoRA scales the update matrix $\Delta W$ by a factor of $\alpha / r$, where $\alpha$ is a hyperparameter. (To keep the formula tidy, we ignore the fact that the linear transformation in the pre-trained model may additionally include a bias.)

In [189]:
import torch.nn as nn

class LoRA(nn.Module):

    def __init__(self, pretrained: nn.Linear, rank=12, alpha=24):
        super().__init__()
        self.rank = rank
        self.scale = alpha / rank
        self.W0 = pretrained # the pretrained model layer

        self.A = nn.Parameter(nn.init.normal_(torch.empty(pretrained.weight.shape[0] , rank)))
        self.B = nn.Parameter(nn.init.zeros_(torch.zeros(rank, pretrained.weight.shape[1])))
        

    def forward(self, x):
        return x @ (self.W0.weight + self.A @ self.B) 

Your code must comply with the following specification:

**__init__** (*self*, *pretrained*, *rank* = 12, *alpha* = 24)

> Initialises the LoRA adapter. This sets up the matrices $A$ and $B$ from the equation above. The matrix $A$ is initialised with random weights from a standard normal distribution; the matrix $B$ is initialised with zeros. The argument *pretrained* is the linear transformation from the pre-trained model that should be adapted. The arguments *rank* and *alpha* are the rank $r$ and the hyperparameter $\alpha$ in the equation above.

**forward** (*self*, *x*)

> Sends an input *x* through the adapter, implementing the equation above.

### Inject the adapter into the pre-trained model

The final step is to construct an adapted model by injecting the LoRA adapters into the pre-trained model.

In [194]:
def make_lora_model(rank: int):
    pretrained = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
    extracted = extract(pretrained)
    for key, value in extracted.items():
        lora = LoRA(value, rank, 12) # set alpha to 12 for the next problem 
        extracted[key] = lora
    return replace(pretrained, extracted)


Implement the function to match the following specification:

> **make_lora_model** (*rank*)
>
> Returns a version of the pre-trained model in which every query and value linear layer is wrapped using a LoRA adapter. The rank of the adapter is specified by the argument *rank*. The *alpha* value of the adapter is set to twice the rank (a common rule of thumb.)

Run the next cell to evaluate your model for $r = 6$ and $\alpha = 12$. How many trainable parameters does the adapted model have? What accuracy do you get? How do these value relate to the number of trainable parameters and accuracy of the fully fine-tuned model, in terms of percentages?

In [191]:
lora_model = make_lora_model(6)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [196]:
trainer = make_trainer(lora_model)
trainer.train()

  1%|          | 5/750 [00:10<21:43,  1.75s/it]

                                         
                                                 
  1%|          | 5/750 [00:10<21:43,  1.75s/it]

{'loss': 0.0035, 'learning_rate': 4.966666666666667e-05, 'epoch': 0.02}


  1%|▏         | 10/750 [00:17<19:21,  1.57s/it]

                                         
                                                 
  1%|▏         | 10/750 [00:17<19:21,  1.57s/it]

{'loss': 0.8834, 'learning_rate': 4.933333333333334e-05, 'epoch': 0.04}


  2%|▏         | 15/750 [00:25<18:36,  1.52s/it]

                                         
                                                 
  2%|▏         | 15/750 [00:25<18:36,  1.52s/it]

...

100%|██████████| 750/750 [20:45<00:00,  1.73s/it]

                                         
                                                 
100%|██████████| 750/750 [20:45<00:00,  1.73s/it]

{'loss': 0.0005, 'learning_rate': 0.0, 'epoch': 3.0}



































































































































                                      

                                         
                                                 


100%|██████████| 750/750 [21:14<00:00,  1.73s/it]



                                         
                                                 
100%|██████████| 750/750 [21:14<00:00,  1.70s/it]

{'eval_loss': 0.9134382605552673, 'eval_accuracy': 0.854, 'eval_runtime': 29.7023, 'eval_samples_per_second': 16.834, 'eval_steps_per_second': 2.121, 'epoch': 3.0}
{'train_runtime': 1274.8991, 'train_samples_per_second': 4.706, 'train_steps_per_second': 0.588, 'train_loss': 0.07740658305771649, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=0.07740658305771649, metrics={'train_runtime': 1274.8991, 'train_samples_per_second': 4.706, 'train_steps_per_second': 0.588, 'train_loss': 0.07740658305771649, 'epoch': 3.0})

# Problem 6: Alternatives to Transformer-based models (6 Points)

Even with methods for parameter-efficient fine-tuning, applying DistilBERT and other Transformer-based models comes at a significant cost – an investment that does not always pay off. In the final problem of this lab, we ask you to explore a more traditional approach to classification and contrast it with the pre-training/fine-tuning approach of neural language models.

Browse the web to find a tutorial on how to apply a classifier from the [scikit-learn](https://scikit-learn.org/stable/) library to the problem of sentiment classification and implement the method here in this notebook. We suggest you use [multinomial Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) or [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). (Once you have code for one method, it is easy to switch to the other.) Evaluate your chosen classifier on the IMDB dataset.

Document your exploration in a short reflection piece (ca. 150 words). Respond to the following prompts:

* Which classifier did you try? What results did you get? How long did it take you to train and run the classifier?
* What is your perspective on the trade-off between accuracy and resource requirements between the two approaches?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

In [197]:
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Load the IMDB dataset
imdb_dataset = load_dataset("imdb", split={"train": "train", "eval": "test"})


# Preprocess the data: Extract reviews and labels, and vectorize the text
def preprocess_data(dataset):
    reviews = [example["text"] for example in dataset]
    labels = [example["label"] for example in dataset]
    return reviews, labels


train_reviews, train_labels = preprocess_data(imdb_dataset["train"])
eval_reviews, eval_labels = preprocess_data(imdb_dataset["eval"])

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_reviews)
X_eval = vectorizer.transform(eval_reviews)

# Train the Multinomial Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, train_labels)

# Evaluate the classifier
pred_labels = clf.predict(X_eval)
print("Accuracy:", accuracy_score(eval_labels, pred_labels))
print(classification_report(eval_labels, pred_labels))


Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 357735.36 examples/s]


Accuracy: 0.81356
              precision    recall  f1-score   support

           0       0.78      0.88      0.82     12500
           1       0.86      0.75      0.80     12500

    accuracy                           0.81     25000
   macro avg       0.82      0.81      0.81     25000
weighted avg       0.82      0.81      0.81     25000



## Short reflection piece:

We tried the multinomial naive bayes classifier on the imdb dataset, the same one we used. It went very quick to train and test, around 20 seconds. The accuracy was around 0.81, which is comparable to the headtuned model, altough lower than the finetuned model.

Running the LoRA gave a greater cost-benefit than running the actual finetuning training, while still beating out the headtuning model.

We learned that neural networks are indeed a very good tool, however they might not be the best tool for every task, there could be many other good methods that might be more specialized. The alternative we tried here is a statistical method and almost performed as well as the headtuned model for a fraction of the cost.

This learning is important as LLMs and neural networks are surrounded by more and more hype in companies and everyday conversation. While they indeed are a powerful tool we should not discard alternative methods just because “AI powered” is a sexy slogan for a company to have